# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 03 · Notebook 02 — From DataFrames to Databases: Mental Model Mapping
**Instructor:** Amir Charkhi  |  **Goal:** Master dataframes to databases.

> Format: theory → implementation → best practices → real-world application.


## 🎯 The Big Picture

You've mastered pandas. You're comfortable with DataFrames. Now we're adding SQL to your toolkit.

**Why both?**
- **Pandas**: In-memory, flexible, great for exploration
- **SQL**: Scalable, persistent, great for production

Think of them as complementary tools:
- Use SQL to **extract and reduce** data from large sources
- Use pandas to **explore and visualize** the reduced data
- Use SQL to **productionize** your proven analyses

In [2]:
# Setup and imports
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Custom SQL display function
from IPython.display import Markdown, display

def show_sql(query):
    """Pretty print SQL queries"""
    display(Markdown(f"```sql\n{query}\n```"))

print("✅ Environment ready!")

✅ Environment ready!


## 📊 Setting Up Our Data Laboratory

We'll use the same retail dataset from Week 1, but now in both pandas AND SQL!

In [3]:
# Create sample retail data (same structure as Week 1)
np.random.seed(42)

# Generate sample data
n_transactions = 10000
n_customers = 1500
n_products = 200

# Create transactions
transactions = pd.DataFrame({
    'transaction_id': range(1, n_transactions + 1),
    'customer_id': np.random.randint(1, n_customers + 1, n_transactions),
    'product_id': np.random.randint(1, n_products + 1, n_transactions),
    'quantity': np.random.randint(1, 5, n_transactions),
    'date': pd.date_range('2024-01-01', periods=n_transactions, freq='15min'),
    'store_id': np.random.choice(['NYC', 'LA', 'CHI', 'HOU', 'PHX'], n_transactions)
})

# Create products
categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Sports']
products = pd.DataFrame({
    'product_id': range(1, n_products + 1),
    'product_name': [f'Product_{i}' for i in range(1, n_products + 1)],
    'category': np.random.choice(categories, n_products),
    'price': np.round(np.random.uniform(10, 500, n_products), 2)
})

# Create customers
customers = pd.DataFrame({
    'customer_id': range(1, n_customers + 1),
    'customer_name': [f'Customer_{i}' for i in range(1, n_customers + 1)],
    'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], n_customers),
    'signup_date': pd.date_range('2023-01-01', periods=n_customers, freq='6H')
})

# Add revenue column
transactions = transactions.merge(products[['product_id', 'price']], on='product_id')
transactions['revenue'] = transactions['quantity'] * transactions['price']

print(f"📦 Created {len(transactions):,} transactions")
print(f"👥 Created {len(customers):,} customers")
print(f"🏷️ Created {len(products):,} products")

# Preview the data
transactions.head()

📦 Created 10,000 transactions
👥 Created 1,500 customers
🏷️ Created 200 products


transaction_id  customer_id  product_id  quantity                date  \
0               1         1127          74         2 2024-01-01 00:00:00   
1               2         1460          98         2 2024-01-01 00:15:00   
2               3          861         154         2 2024-01-01 00:30:00   
3               4         1295         158         3 2024-01-01 00:45:00   
4               5         1131         110         4 2024-01-01 01:00:00   

  store_id  price  revenue  
0      NYC 264.05   528.10  
1      CHI 338.38   676.76  
2      PHX 441.58   883.16  
3      NYC 267.93   803.79  
4      HOU 187.62   750.48

In [4]:
# Create SQLite database and load our data
conn = sqlite3.connect('retail.db')

# Load data into SQL
transactions.to_sql('transactions', conn, if_exists='replace', index=False)
products.to_sql('products', conn, if_exists='replace', index=False)
customers.to_sql('customers', conn, if_exists='replace', index=False)

# Create a SQLAlchemy engine for pandas integration
engine = create_engine('sqlite:///retail.db')

print("✅ Database created and data loaded!")

✅ Database created and data loaded!


In [5]:
# Verify tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("\n📊 Tables in database:")
for table in tables['name']:
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0, 0]
    print(f"  - {table}: {count:,} rows")


📊 Tables in database:
  - transactions: 10,000 rows
  - products: 200 rows
  - customers: 1,500 rows


---

## 🔄 Part 1: Basic Operations - SELECT, WHERE, ORDER BY

Let's start with the fundamentals. Every pandas operation has a SQL equivalent!

### 1.1 Selecting Columns

The most basic operation - choosing which columns to work with.

In [17]:
# PANDAS: Select specific columns
pandas_result = transactions[['transaction_id', 'customer_id', 'revenue']].head()
print("🐼 Pandas approach:")
print(pandas_result)

print("\n" + "="*50 + "\n")

# SQL: Select specific columns
sql_query = """
SELECT transaction_id, customer_id, revenue
FROM transactions
LIMIT 5
"""

print("🗄️ SQL approach:")
show_sql(sql_query)
sql_result = pd.read_sql(sql_query, conn)
print(sql_result)

print("\n✅ Results are identical!")

🐼 Pandas approach:
   transaction_id  customer_id  revenue
0               1         1127   528.10
1               2         1460   676.76
2               3          861   883.16
3               4         1295   803.79
4               5         1131   750.48


🗄️ SQL approach:


```sql

SELECT transaction_id, customer_id, revenue
FROM transactions
LIMIT 5

```

   transaction_id  customer_id  revenue
0               1         1127   528.10
1               2         1460   676.76
2               3          861   883.16
3               4         1295   803.79
4               5         1131   750.48

✅ Results are identical!


### 1.2 Filtering Rows (WHERE clause)

Filtering is where SQL starts to shine with complex conditions.

In [18]:
# PANDAS: Multiple filter conditions
pandas_filter = transactions[
    (transactions['revenue'] > 500) & 
    (transactions['store_id'] == 'NYC')
][['transaction_id', 'revenue', 'store_id']].head()

print("🐼 Pandas filtering:")
print("df[(df['revenue'] > 500) & (df['store_id'] == 'NYC')]")
print(pandas_filter)

print("\n" + "="*50 + "\n")

# SQL: WHERE clause
sql_query = """
SELECT transaction_id, revenue, store_id
FROM transactions
WHERE revenue > 500 
  AND store_id = 'NYC'
LIMIT 5
"""

print("🗄️ SQL filtering:")
show_sql(sql_query)
sql_filter = pd.read_sql(sql_query, conn)
print(sql_filter)

# Pro tip comparison
print("\n💡 Pro Tip: SQL WHERE is often more readable for complex conditions!")

🐼 Pandas filtering:
df[(df['revenue'] > 500) & (df['store_id'] == 'NYC')]
    transaction_id  revenue store_id
0                1   528.10      NYC
3                4   803.79      NYC
7                8   777.34      NYC
25              26  1199.04      NYC
30              31  1324.74      NYC


🗄️ SQL filtering:


```sql

SELECT transaction_id, revenue, store_id
FROM transactions
WHERE revenue > 500 
  AND store_id = 'NYC'
LIMIT 5

```

   transaction_id  revenue store_id
0               1   528.10      NYC
1               4   803.79      NYC
2               8   777.34      NYC
3              26  1199.04      NYC
4              31  1324.74      NYC

💡 Pro Tip: SQL WHERE is often more readable for complex conditions!


### 1.3 Sorting (ORDER BY)

Sorting is fundamental for rankings and time series analysis.

In [19]:
# PANDAS: Sort by multiple columns
pandas_sorted = transactions.nlargest(10, 'revenue')[['transaction_id', 'customer_id', 'revenue']]

print("🐼 Pandas sorting (top 10 by revenue):")
print("df.nlargest(10, 'revenue')")
print(pandas_sorted)

print("\n" + "="*50 + "\n")

# SQL: ORDER BY
sql_query = """
SELECT transaction_id, customer_id, revenue
FROM transactions
ORDER BY revenue DESC
LIMIT 10
"""

print("🗄️ SQL sorting:")
show_sql(sql_query)
sql_sorted = pd.read_sql(sql_query, conn)
print(sql_sorted)

🐼 Pandas sorting (top 10 by revenue):
df.nlargest(10, 'revenue')
      transaction_id  customer_id  revenue
1268            1269          285  1933.76
1876            1877          166  1933.76
2045            2046         1187  1933.76
2297            2298         1050  1933.76
4599            4600          121  1933.76
8809            8810          728  1933.76
8836            8837           60  1933.76
8894            8895         1158  1933.76
9579            9580          466  1933.76
9739            9740          376  1933.76


🗄️ SQL sorting:


```sql

SELECT transaction_id, customer_id, revenue
FROM transactions
ORDER BY revenue DESC
LIMIT 10

```

   transaction_id  customer_id  revenue
0            1269          285  1933.76
1            1877          166  1933.76
2            2046         1187  1933.76
3            2298         1050  1933.76
4            4600          121  1933.76
5            8810          728  1933.76
6            8837           60  1933.76
7            8895         1158  1933.76
8            9580          466  1933.76
9            9740          376  1933.76


---

## 🔗 Part 2: Aggregations - GROUP BY

This is where the mental models really start to connect!

In [20]:
transactions.columns

Index(['transaction_id', 'customer_id', 'product_id', 'quantity', 'date',
       'store_id', 'price', 'revenue'],
      dtype='object')

### 2.1 Simple Aggregation

In [21]:
# PANDAS: Group by store and calculate metrics
pandas_agg = transactions.groupby('store_id').agg({
    'revenue': ['sum', 'mean', 'count']
}).round(2)

print("🐼 Pandas aggregation:")
print("df.groupby('store_id').agg({'revenue': ['sum', 'mean', 'count']})")
print(pandas_agg)

print("\n" + "="*50 + "\n")

# SQL: GROUP BY with multiple aggregations
sql_query = """
SELECT 
    store_id,
    SUM(revenue) as revenue_sum,
    AVG(revenue) as revenue_mean,
    COUNT(*) as revenue_count
FROM transactions
GROUP BY store_id
ORDER BY revenue_sum DESC
"""

print("🗄️ SQL aggregation:")
show_sql(sql_query)
sql_agg = pd.read_sql(sql_query, conn)
print(sql_agg)

🐼 Pandas aggregation:
df.groupby('store_id').agg({'revenue': ['sum', 'mean', 'count']})
            revenue             
                sum   mean count
store_id                        
CHI      1257251.81 622.09  2021
HOU      1272553.55 628.73  2024
LA       1194809.67 628.85  1900
NYC      1193083.10 625.31  1908
PHX      1311829.97 611.01  2147


🗄️ SQL aggregation:


```sql

SELECT 
    store_id,
    SUM(revenue) as revenue_sum,
    AVG(revenue) as revenue_mean,
    COUNT(*) as revenue_count
FROM transactions
GROUP BY store_id
ORDER BY revenue_sum DESC

```

  store_id  revenue_sum  revenue_mean  revenue_count
0      PHX   1311829.97        611.01           2147
1      HOU   1272553.55        628.73           2024
2      CHI   1257251.81        622.09           2021
3       LA   1194809.67        628.85           1900
4      NYC   1193083.10        625.31           1908


### 2.2 Multiple Grouping Columns

In [6]:
# Add date components for better grouping
transactions['date_only'] = transactions['date'].dt.date
transactions['hour'] = transactions['date'].dt.hour

# Update SQL table
transactions.to_sql('transactions', conn, if_exists='replace', index=False)

# PANDAS: Multi-level groupby
pandas_multi = transactions.groupby(['store_id', 'date_only'])['revenue'].sum().head(10)

print("🐼 Pandas multi-level groupby:")
print("df.groupby(['store_id', 'date_only'])['revenue'].sum()")
print(pandas_multi)

print("\n" + "="*50 + "\n")

# SQL: Multiple GROUP BY columns
sql_query = """
SELECT 
    store_id,
    date_only,
    SUM(revenue) as total_revenue
FROM transactions
GROUP BY store_id, date_only
ORDER BY store_id, date_only
LIMIT 10
"""

print("🗄️ SQL multi-level groupby:")
show_sql(sql_query)
sql_multi = pd.read_sql(sql_query, conn)
print(sql_multi)

🐼 Pandas multi-level groupby:
df.groupby(['store_id', 'date_only'])['revenue'].sum()
store_id  date_only 
CHI       2024-01-01   15293.12
          2024-01-02   11716.84
          2024-01-03   10267.87
          2024-01-04    8972.93
          2024-01-05   11157.58
          2024-01-06   15685.74
          2024-01-07   18873.03
          2024-01-08   11635.85
          2024-01-09    7796.28
          2024-01-10   14762.60
Name: revenue, dtype: float64


🗄️ SQL multi-level groupby:


```sql

SELECT 
    store_id,
    date_only,
    SUM(revenue) as total_revenue
FROM transactions
GROUP BY store_id, date_only
ORDER BY store_id, date_only
LIMIT 10

```

  store_id   date_only  total_revenue
0      CHI  2024-01-01       15293.12
1      CHI  2024-01-02       11716.84
2      CHI  2024-01-03       10267.87
3      CHI  2024-01-04        8972.93
4      CHI  2024-01-05       11157.58
5      CHI  2024-01-06       15685.74
6      CHI  2024-01-07       18873.03
7      CHI  2024-01-08       11635.85
8      CHI  2024-01-09        7796.28
9      CHI  2024-01-10       14762.60


### 2.3 Filtering After Aggregation (HAVING clause)

This is a key concept - filtering AFTER grouping!

In [23]:
# PANDAS: Filter after groupby
store_totals = transactions.groupby('store_id')['revenue'].sum()
pandas_having = store_totals[store_totals > 100000]

print("🐼 Pandas approach (filter after groupby):")
print("grouped = df.groupby('store_id')['revenue'].sum()")
print("grouped[grouped > 100000]")
print(pandas_having)

print("\n" + "="*50 + "\n")

# SQL: HAVING clause
sql_query = """
SELECT 
    store_id,
    SUM(revenue) as total_revenue
FROM transactions
GROUP BY store_id
HAVING SUM(revenue) > 100000
ORDER BY total_revenue DESC
"""

print("🗄️ SQL approach (HAVING clause):")
show_sql(sql_query)
sql_having = pd.read_sql(sql_query, conn)
print(sql_having)

print("\n💡 Key Insight: WHERE filters rows BEFORE grouping, HAVING filters AFTER grouping!")

🐼 Pandas approach (filter after groupby):
grouped = df.groupby('store_id')['revenue'].sum()
grouped[grouped > 100000]
store_id
CHI   1257251.81
HOU   1272553.55
LA    1194809.67
NYC   1193083.10
PHX   1311829.97
Name: revenue, dtype: float64


🗄️ SQL approach (HAVING clause):


```sql

SELECT 
    store_id,
    SUM(revenue) as total_revenue
FROM transactions
GROUP BY store_id
HAVING SUM(revenue) > 100000
ORDER BY total_revenue DESC

```

  store_id  total_revenue
0      PHX     1311829.97
1      HOU     1272553.55
2      CHI     1257251.81
3       LA     1194809.67
4      NYC     1193083.10

💡 Key Insight: WHERE filters rows BEFORE grouping, HAVING filters AFTER grouping!


---

## 🔗 Part 3: JOINs - Combining Tables

JOINs are SQL's superpower. Let's map them to pandas merge operations!

### 3.1 Inner Join (Default)

In [24]:
# PANDAS: Inner join
pandas_inner = transactions[['transaction_id', 'customer_id', 'product_id', 'revenue']].merge(
    products[['product_id', 'product_name', 'category']],
    on='product_id',
    how='inner'
).head()

print("🐼 Pandas inner join:")
print("df1.merge(df2, on='product_id', how='inner')")
print(pandas_inner)

print("\n" + "="*50 + "\n")

# SQL: INNER JOIN
sql_query = """
SELECT 
    t.transaction_id,
    t.customer_id,
    t.product_id,
    t.revenue,
    p.product_name,
    p.category
FROM transactions t
INNER JOIN products p ON t.product_id = p.product_id
LIMIT 5
"""

print("🗄️ SQL inner join:")
show_sql(sql_query)
sql_inner = pd.read_sql(sql_query, conn)
print(sql_inner)

🐼 Pandas inner join:
df1.merge(df2, on='product_id', how='inner')
   transaction_id  customer_id  product_id  revenue product_name category
0               1         1127          74   528.10   Product_74     Food
1               2         1460          98   676.76   Product_98    Books
2               3          861         154   883.16  Product_154     Food
3               4         1295         158   803.79  Product_158   Sports
4               5         1131         110   750.48  Product_110   Sports


🗄️ SQL inner join:


```sql

SELECT 
    t.transaction_id,
    t.customer_id,
    t.product_id,
    t.revenue,
    p.product_name,
    p.category
FROM transactions t
INNER JOIN products p ON t.product_id = p.product_id
LIMIT 5

```

   transaction_id  customer_id  product_id  revenue product_name category
0               1         1127          74   528.10   Product_74     Food
1               2         1460          98   676.76   Product_98    Books
2               3          861         154   883.16  Product_154     Food
3               4         1295         158   803.79  Product_158   Sports
4               5         1131         110   750.48  Product_110   Sports


### 3.2 Left Join

In [25]:
# Create some customers without transactions for demonstration
all_customers = customers[['customer_id', 'customer_name']].head(10)
customer_revenue = transactions.groupby('customer_id')['revenue'].sum().reset_index()

# PANDAS: Left join
pandas_left = all_customers.merge(
    customer_revenue,
    on='customer_id',
    how='left'
).fillna(0)

print("🐼 Pandas left join (all customers, even without purchases):")
print("customers.merge(revenue, on='customer_id', how='left').fillna(0)")
print(pandas_left)

print("\n" + "="*50 + "\n")

# SQL: LEFT JOIN
sql_query = """
SELECT 
    c.customer_id,
    c.customer_name,
    COALESCE(SUM(t.revenue), 0) as total_revenue
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
WHERE c.customer_id <= 10
GROUP BY c.customer_id, c.customer_name
ORDER BY c.customer_id
"""

print("🗄️ SQL left join:")
show_sql(sql_query)
sql_left = pd.read_sql(sql_query, conn)
print(sql_left)

print("\n💡 COALESCE in SQL = fillna in pandas!")

🐼 Pandas left join (all customers, even without purchases):
customers.merge(revenue, on='customer_id', how='left').fillna(0)
   customer_id customer_name  revenue
0            1    Customer_1  5176.93
1            2    Customer_2  2232.51
2            3    Customer_3  1569.18
3            4    Customer_4  2479.04
4            5    Customer_5  3616.14
5            6    Customer_6  3528.33
6            7    Customer_7  4061.29
7            8    Customer_8  2230.96
8            9    Customer_9  2309.34
9           10   Customer_10  5605.37


🗄️ SQL left join:


```sql

SELECT 
    c.customer_id,
    c.customer_name,
    COALESCE(SUM(t.revenue), 0) as total_revenue
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
WHERE c.customer_id <= 10
GROUP BY c.customer_id, c.customer_name
ORDER BY c.customer_id

```

   customer_id customer_name  total_revenue
0            1    Customer_1        5176.93
1            2    Customer_2        2232.51
2            3    Customer_3        1569.18
3            4    Customer_4        2479.04
4            5    Customer_5        3616.14
5            6    Customer_6        3528.33
6            7    Customer_7        4061.29
7            8    Customer_8        2230.96
8            9    Customer_9        2309.34
9           10   Customer_10        5605.37

💡 COALESCE in SQL = fillna in pandas!


### 3.3 Multiple Joins

In [26]:
# PANDAS: Chain multiple merges
pandas_multi_join = transactions[['transaction_id', 'customer_id', 'product_id', 'revenue']].merge(
    products[['product_id', 'product_name', 'category']],
    on='product_id'
).merge(
    customers[['customer_id', 'customer_name', 'city']],
    on='customer_id'
).head()

print("🐼 Pandas multiple joins:")
print("df.merge(products, on='product_id').merge(customers, on='customer_id')")
print(pandas_multi_join)

print("\n" + "="*50 + "\n")

# SQL: Multiple JOINs
sql_query = """
SELECT 
    t.transaction_id,
    c.customer_name,
    c.city,
    p.product_name,
    p.category,
    t.revenue
FROM transactions t
JOIN products p ON t.product_id = p.product_id
JOIN customers c ON t.customer_id = c.customer_id
LIMIT 5
"""

print("🗄️ SQL multiple joins:")
show_sql(sql_query)
sql_multi_join = pd.read_sql(sql_query, conn)
print(sql_multi_join)

🐼 Pandas multiple joins:
df.merge(products, on='product_id').merge(customers, on='customer_id')
   transaction_id  customer_id  product_id  revenue product_name category  \
0               1         1127          74   528.10   Product_74     Food   
1               2         1460          98   676.76   Product_98    Books   
2               3          861         154   883.16  Product_154     Food   
3               4         1295         158   803.79  Product_158   Sports   
4               5         1131         110   750.48  Product_110   Sports   

   customer_name         city  
0  Customer_1127     New York  
1  Customer_1460  Los Angeles  
2   Customer_861      Chicago  
3  Customer_1295      Houston  
4  Customer_1131      Phoenix  


🗄️ SQL multiple joins:


```sql

SELECT 
    t.transaction_id,
    c.customer_name,
    c.city,
    p.product_name,
    p.category,
    t.revenue
FROM transactions t
JOIN products p ON t.product_id = p.product_id
JOIN customers c ON t.customer_id = c.customer_id
LIMIT 5

```

   transaction_id  customer_name         city product_name category  revenue
0               1  Customer_1127     New York   Product_74     Food   528.10
1               2  Customer_1460  Los Angeles   Product_98    Books   676.76
2               3   Customer_861      Chicago  Product_154     Food   883.16
3               4  Customer_1295      Houston  Product_158   Sports   803.79
4               5  Customer_1131      Phoenix  Product_110   Sports   750.48


---

## 🚀 Part 4: Advanced Operations - Window Functions

Window functions are incredibly powerful for analytics. Let's see how pandas and SQL compare!

### 4.1 Ranking Functions

In [27]:
# PANDAS: Ranking within groups
pandas_rank = transactions.copy()
pandas_rank['rank_in_store'] = pandas_rank.groupby('store_id')['revenue'].rank(method='dense', ascending=False)
top_per_store = pandas_rank[pandas_rank['rank_in_store'] <= 3][['store_id', 'transaction_id', 'revenue', 'rank_in_store']].sort_values(['store_id', 'rank_in_store'])

print("🐼 Pandas ranking (top 3 transactions per store):")
print("df['rank'] = df.groupby('store_id')['revenue'].rank(method='dense', ascending=False)")
print(top_per_store.head(10))

print("\n" + "="*50 + "\n")

# SQL: Window function with RANK()
sql_query = """
WITH ranked_transactions AS (
    SELECT 
        store_id,
        transaction_id,
        revenue,
        DENSE_RANK() OVER (PARTITION BY store_id ORDER BY revenue DESC) as rank_in_store
    FROM transactions
)
SELECT *
FROM ranked_transactions
WHERE rank_in_store <= 3
ORDER BY store_id, rank_in_store
LIMIT 10
"""

print("🗄️ SQL window function:")
show_sql(sql_query)
sql_rank = pd.read_sql(sql_query, conn)
print(sql_rank)

🐼 Pandas ranking (top 3 transactions per store):
df['rank'] = df.groupby('store_id')['revenue'].rank(method='dense', ascending=False)
     store_id  transaction_id  revenue  rank_in_store
1268      CHI            1269  1933.76           1.00
9739      CHI            9740  1933.76           1.00
2227      CHI            2228  1932.04           2.00
2403      CHI            2404  1932.04           2.00
5631      CHI            5632  1932.04           2.00
1512      CHI            1513  1928.48           3.00
3703      CHI            3704  1928.48           3.00
6042      CHI            6043  1928.48           3.00
7441      CHI            7442  1928.48           3.00
8019      CHI            8020  1928.48           3.00


🗄️ SQL window function:


```sql

WITH ranked_transactions AS (
    SELECT 
        store_id,
        transaction_id,
        revenue,
        DENSE_RANK() OVER (PARTITION BY store_id ORDER BY revenue DESC) as rank_in_store
    FROM transactions
)
SELECT *
FROM ranked_transactions
WHERE rank_in_store <= 3
ORDER BY store_id, rank_in_store
LIMIT 10

```

  store_id  transaction_id  revenue  rank_in_store
0      CHI            1269  1933.76              1
1      CHI            9740  1933.76              1
2      CHI            2228  1932.04              2
3      CHI            2404  1932.04              2
4      CHI            5632  1932.04              2
5      CHI            1513  1928.48              3
6      CHI            3704  1928.48              3
7      CHI            6043  1928.48              3
8      CHI            7442  1928.48              3
9      CHI            8020  1928.48              3


### 4.2 Running Totals and Moving Averages

In [28]:
# PANDAS: Cumulative sum and rolling average
daily_revenue = transactions.groupby('date_only')['revenue'].sum().reset_index()
daily_revenue = daily_revenue.sort_values('date_only')
daily_revenue['cumulative_revenue'] = daily_revenue['revenue'].cumsum()
daily_revenue['moving_avg_7d'] = daily_revenue['revenue'].rolling(window=7, min_periods=1).mean()

print("🐼 Pandas cumulative and rolling:")
print("df['cumsum'] = df['revenue'].cumsum()")
print("df['rolling_avg'] = df['revenue'].rolling(window=7).mean()")
print(daily_revenue.head(10))

print("\n" + "="*50 + "\n")

# SQL: Window functions for running totals
sql_query = """
WITH daily_totals AS (
    SELECT 
        date_only,
        SUM(revenue) as daily_revenue
    FROM transactions
    GROUP BY date_only
)
SELECT 
    date_only,
    daily_revenue,
    SUM(daily_revenue) OVER (ORDER BY date_only) as cumulative_revenue,
    AVG(daily_revenue) OVER (
        ORDER BY date_only 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as moving_avg_7d
FROM daily_totals
ORDER BY date_only
LIMIT 10
"""

print("🗄️ SQL window functions:")
show_sql(sql_query)
sql_window = pd.read_sql(sql_query, conn)
print(sql_window)

🐼 Pandas cumulative and rolling:
df['cumsum'] = df['revenue'].cumsum()
df['rolling_avg'] = df['revenue'].rolling(window=7).mean()
    date_only  revenue  cumulative_revenue  moving_avg_7d
0  2024-01-01 59709.54            59709.54       59709.54
1  2024-01-02 59637.14           119346.68       59673.34
2  2024-01-03 52733.77           172080.45       57360.15
3  2024-01-04 62495.57           234576.02       58644.01
4  2024-01-05 57646.39           292222.41       58444.48
5  2024-01-06 61774.59           353997.00       58999.50
6  2024-01-07 62692.86           416689.86       59527.12
7  2024-01-08 63542.02           480231.88       60074.62
8  2024-01-09 59077.97           539309.85       59994.74
9  2024-01-10 57337.61           596647.46       60652.43


🗄️ SQL window functions:


```sql

WITH daily_totals AS (
    SELECT 
        date_only,
        SUM(revenue) as daily_revenue
    FROM transactions
    GROUP BY date_only
)
SELECT 
    date_only,
    daily_revenue,
    SUM(daily_revenue) OVER (ORDER BY date_only) as cumulative_revenue,
    AVG(daily_revenue) OVER (
        ORDER BY date_only 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as moving_avg_7d
FROM daily_totals
ORDER BY date_only
LIMIT 10

```

    date_only  daily_revenue  cumulative_revenue  moving_avg_7d
0  2024-01-01       59709.54            59709.54       59709.54
1  2024-01-02       59637.14           119346.68       59673.34
2  2024-01-03       52733.77           172080.45       57360.15
3  2024-01-04       62495.57           234576.02       58644.01
4  2024-01-05       57646.39           292222.41       58444.48
5  2024-01-06       61774.59           353997.00       58999.50
6  2024-01-07       62692.86           416689.86       59527.12
7  2024-01-08       63542.02           480231.88       60074.62
8  2024-01-09       59077.97           539309.85       59994.74
9  2024-01-10       57337.61           596647.46       60652.43


### 4.3 Lead and Lag Operations

In [29]:
# PANDAS: Shift operations for time series
daily_revenue = transactions.groupby('date_only')['revenue'].sum().reset_index().sort_values('date_only')
daily_revenue['prev_day_revenue'] = daily_revenue['revenue'].shift(1)
daily_revenue['next_day_revenue'] = daily_revenue['revenue'].shift(-1)
daily_revenue['day_over_day_change'] = daily_revenue['revenue'] - daily_revenue['prev_day_revenue']

print("🐼 Pandas shift operations:")
print("df['prev'] = df['revenue'].shift(1)")
print("df['next'] = df['revenue'].shift(-1)")
print(daily_revenue.head(10))

print("\n" + "="*50 + "\n")

# SQL: LAG and LEAD functions
sql_query = """
WITH daily_totals AS (
    SELECT 
        date_only,
        SUM(revenue) as daily_revenue
    FROM transactions
    GROUP BY date_only
)
SELECT 
    date_only,
    daily_revenue,
    LAG(daily_revenue, 1) OVER (ORDER BY date_only) as prev_day_revenue,
    LEAD(daily_revenue, 1) OVER (ORDER BY date_only) as next_day_revenue,
    daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY date_only) as day_over_day_change
FROM daily_totals
ORDER BY date_only
LIMIT 10
"""

print("🗄️ SQL LAG/LEAD:")
show_sql(sql_query)
sql_lag_lead = pd.read_sql(sql_query, conn)
print(sql_lag_lead)

🐼 Pandas shift operations:
df['prev'] = df['revenue'].shift(1)
df['next'] = df['revenue'].shift(-1)
    date_only  revenue  prev_day_revenue  next_day_revenue  \
0  2024-01-01 59709.54               NaN          59637.14   
1  2024-01-02 59637.14          59709.54          52733.77   
2  2024-01-03 52733.77          59637.14          62495.57   
3  2024-01-04 62495.57          52733.77          57646.39   
4  2024-01-05 57646.39          62495.57          61774.59   
5  2024-01-06 61774.59          57646.39          62692.86   
6  2024-01-07 62692.86          61774.59          63542.02   
7  2024-01-08 63542.02          62692.86          59077.97   
8  2024-01-09 59077.97          63542.02          57337.61   
9  2024-01-10 57337.61          59077.97          61057.75   

   day_over_day_change  
0                  NaN  
1               -72.40  
2             -6903.37  
3              9761.80  
4             -4849.18  
5              4128.20  
6               918.27  
7               8

```sql

WITH daily_totals AS (
    SELECT 
        date_only,
        SUM(revenue) as daily_revenue
    FROM transactions
    GROUP BY date_only
)
SELECT 
    date_only,
    daily_revenue,
    LAG(daily_revenue, 1) OVER (ORDER BY date_only) as prev_day_revenue,
    LEAD(daily_revenue, 1) OVER (ORDER BY date_only) as next_day_revenue,
    daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY date_only) as day_over_day_change
FROM daily_totals
ORDER BY date_only
LIMIT 10

```

    date_only  daily_revenue  prev_day_revenue  next_day_revenue  \
0  2024-01-01       59709.54               NaN          59637.14   
1  2024-01-02       59637.14          59709.54          52733.77   
2  2024-01-03       52733.77          59637.14          62495.57   
3  2024-01-04       62495.57          52733.77          57646.39   
4  2024-01-05       57646.39          62495.57          61774.59   
5  2024-01-06       61774.59          57646.39          62692.86   
6  2024-01-07       62692.86          61774.59          63542.02   
7  2024-01-08       63542.02          62692.86          59077.97   
8  2024-01-09       59077.97          63542.02          57337.61   
9  2024-01-10       57337.61          59077.97          61057.75   

   day_over_day_change  
0                  NaN  
1               -72.40  
2             -6903.37  
3              9761.80  
4             -4849.18  
5              4128.20  
6               918.27  
7               849.16  
8             -4464.05  
9

---

## 🔄 Part 5: Subqueries and CTEs

Complex analytical questions often require multiple steps. Let's see how to structure them!

### 5.1 Subqueries vs Method Chaining

In [32]:
# PANDAS: Method chaining for complex logic
# Find customers whose average order is above the overall average
overall_avg = transactions['revenue'].mean()

pandas_complex = (
    transactions
    .groupby('customer_id')['revenue']
    .mean()
    .reset_index()
    .rename(columns={'revenue': 'avg_revenue'})
    .query(f'avg_revenue > {overall_avg}')
    .sort_values('avg_revenue', ascending=False)
    .head(10)
)

print(f"🐼 Pandas: Customers with avg order > ${overall_avg:.2f}")
print(pandas_complex)

print("\n" + "="*50 + "\n")

# SQL: Using subquery
sql_query = """
SELECT 
    customer_id,
    AVG(revenue) as avg_revenue
FROM transactions
GROUP BY customer_id
HAVING AVG(revenue) > (
    SELECT AVG(revenue) 
    FROM transactions
)
ORDER BY avg_revenue DESC
LIMIT 10
"""

print("🗄️ SQL with subquery:")
show_sql(sql_query)
sql_subquery = pd.read_sql(sql_query, conn)
print(sql_subquery)

🐼 Pandas: Customers with avg order > $622.95
     customer_id  avg_revenue
318          319      1905.36
296          297      1397.41
120          121      1392.62
967          968      1375.14
842          843      1264.84
635          636      1252.44
43            44      1249.63
97            98      1237.95
417          418      1216.89
167          168      1214.04


🗄️ SQL with subquery:


```sql

SELECT 
    customer_id,
    AVG(revenue) as avg_revenue
FROM transactions
GROUP BY customer_id
HAVING AVG(revenue) > (
    SELECT AVG(revenue) 
    FROM transactions
)
ORDER BY avg_revenue DESC
LIMIT 10

```

   customer_id  avg_revenue
0          319      1905.36
1          297      1397.41
2          121      1392.62
3          968      1375.14
4          843      1264.84
5          636      1252.44
6           44      1249.63
7           98      1237.95
8          418      1216.89
9          168      1214.04


### 5.2 Common Table Expressions (CTEs)

CTEs are like creating temporary DataFrames in your SQL query!

In [33]:
# PANDAS: Multi-step analysis
# Step 1: Calculate customer metrics
customer_metrics = transactions.groupby('customer_id').agg({
    'revenue': ['sum', 'mean', 'count']
}).round(2)
customer_metrics.columns = ['total_revenue', 'avg_revenue', 'transaction_count']
customer_metrics = customer_metrics.reset_index()

# Step 2: Categorize customers
customer_metrics['customer_segment'] = pd.cut(
    customer_metrics['total_revenue'],
    bins=[0, 1000, 5000, float('inf')],
    labels=['Low', 'Medium', 'High']
)

# Step 3: Summary by segment
segment_summary = customer_metrics.groupby('customer_segment').agg({
    'customer_id': 'count',
    'total_revenue': 'mean'
}).round(2)

print("🐼 Pandas multi-step analysis:")
print(segment_summary)

print("\n" + "="*50 + "\n")

# SQL: Using CTEs for the same analysis
sql_query = """
WITH customer_metrics AS (
    SELECT 
        customer_id,
        SUM(revenue) as total_revenue,
        AVG(revenue) as avg_revenue,
        COUNT(*) as transaction_count
    FROM transactions
    GROUP BY customer_id
),
customer_segments AS (
    SELECT 
        *,
        CASE 
            WHEN total_revenue <= 1000 THEN 'Low'
            WHEN total_revenue <= 5000 THEN 'Medium'
            ELSE 'High'
        END as customer_segment
    FROM customer_metrics
)
SELECT 
    customer_segment,
    COUNT(*) as customer_count,
    AVG(total_revenue) as avg_segment_revenue
FROM customer_segments
GROUP BY customer_segment
ORDER BY avg_segment_revenue
"""

print("🗄️ SQL with CTEs:")
show_sql(sql_query)
sql_cte = pd.read_sql(sql_query, conn)
print(sql_cte)

print("\n💡 CTEs make complex SQL queries readable and modular, just like method chaining in pandas!")

🐼 Pandas multi-step analysis:
                  customer_id  total_revenue
customer_segment                            
Low                        46         657.00
Medium                    989        3223.83
High                      463        6503.10


🗄️ SQL with CTEs:


```sql

WITH customer_metrics AS (
    SELECT 
        customer_id,
        SUM(revenue) as total_revenue,
        AVG(revenue) as avg_revenue,
        COUNT(*) as transaction_count
    FROM transactions
    GROUP BY customer_id
),
customer_segments AS (
    SELECT 
        *,
        CASE 
            WHEN total_revenue <= 1000 THEN 'Low'
            WHEN total_revenue <= 5000 THEN 'Medium'
            ELSE 'High'
        END as customer_segment
    FROM customer_metrics
)
SELECT 
    customer_segment,
    COUNT(*) as customer_count,
    AVG(total_revenue) as avg_segment_revenue
FROM customer_segments
GROUP BY customer_segment
ORDER BY avg_segment_revenue

```

  customer_segment  customer_count  avg_segment_revenue
0              Low              46               657.00
1           Medium             989              3223.83
2             High             463              6503.10

💡 CTEs make complex SQL queries readable and modular, just like method chaining in pandas!


---

## 🚀 Part 6: Performance Considerations

When should you use pandas vs SQL? Let's understand the trade-offs!

In [34]:
import time

# Test 1: Simple filtering
print("📊 Test 1: Simple filtering (revenue > 100)\n")

# Pandas timing
start = time.time()
pandas_result = transactions[transactions['revenue'] > 100]
pandas_time = time.time() - start
print(f"🐼 Pandas: {len(pandas_result):,} rows in {pandas_time:.4f} seconds")

# SQL timing
start = time.time()
sql_result = pd.read_sql("SELECT * FROM transactions WHERE revenue > 100", conn)
sql_time = time.time() - start
print(f"🗄️ SQL: {len(sql_result):,} rows in {sql_time:.4f} seconds")

print(f"\n⚡ Faster: {'Pandas' if pandas_time < sql_time else 'SQL'} by {abs(pandas_time - sql_time):.4f}s")

print("\n" + "="*50 + "\n")

# Test 2: Complex aggregation
print("📊 Test 2: Complex aggregation (group by store, calculate multiple metrics)\n")

# Pandas timing
start = time.time()
pandas_agg = transactions.groupby('store_id').agg({
    'revenue': ['sum', 'mean', 'std'],
    'quantity': ['sum', 'mean'],
    'transaction_id': 'count'
})
pandas_time = time.time() - start
print(f"🐼 Pandas: Aggregated in {pandas_time:.4f} seconds")

# SQL timing
start = time.time()
sql_agg = pd.read_sql("""
    SELECT 
        store_id,
        SUM(revenue) as revenue_sum,
        AVG(revenue) as revenue_mean,
        SUM(quantity) as quantity_sum,
        AVG(quantity) as quantity_mean,
        COUNT(*) as transaction_count
    FROM transactions
    GROUP BY store_id
""", conn)
sql_time = time.time() - start
print(f"🗄️ SQL: Aggregated in {sql_time:.4f} seconds")

print(f"\n⚡ Faster: {'Pandas' if pandas_time < sql_time else 'SQL'} by {abs(pandas_time - sql_time):.4f}s")

📊 Test 1: Simple filtering (revenue > 100)

🐼 Pandas: 9,340 rows in 0.0026 seconds
🗄️ SQL: 9,340 rows in 0.0313 seconds

⚡ Faster: Pandas by 0.0287s


📊 Test 2: Complex aggregation (group by store, calculate multiple metrics)

🐼 Pandas: Aggregated in 0.0061 seconds
🗄️ SQL: Aggregated in 0.0050 seconds

⚡ Faster: SQL by 0.0010s


### 📊 When to Use Each Tool

Based on our experiments and real-world experience:

In [35]:
decision_matrix = pd.DataFrame({
    'Scenario': [
        'Data exploration & prototyping',
        'Production data pipelines',
        'Complex statistical analysis',
        'Data > 1GB',
        'Real-time dashboards',
        'Ad-hoc business queries',
        'Machine learning features',
        'Data validation & cleaning',
        'Time series manipulation',
        'Joining multiple large tables'
    ],
    'Preferred Tool': [
        '🐼 Pandas',
        '🗄️ SQL',
        '🐼 Pandas',
        '🗄️ SQL',
        '🗄️ SQL',
        '🗄️ SQL',
        '🐼 Pandas → SQL',
        '🐼 Pandas',
        '🐼 Pandas',
        '🗄️ SQL'
    ],
    'Reason': [
        'Interactive, flexible, great for iteration',
        'Scalable, auditable, version-controlled',
        'Rich statistical libraries (scipy, statsmodels)',
        'Memory constraints, let database do the work',
        'Direct queries, no data movement',
        'Standard language, shareable queries',
        'Prototype in pandas, productionize in SQL',
        'Better string/regex operations',
        'Superior datetime handling',
        'Optimized query planner'
    ]
})

print("🎯 DECISION MATRIX: When to Use Pandas vs SQL\n")
for _, row in decision_matrix.iterrows():
    print(f"{row['Preferred Tool']} {row['Scenario']}")
    print(f"    → {row['Reason']}\n")

🎯 DECISION MATRIX: When to Use Pandas vs SQL

🐼 Pandas Data exploration & prototyping
    → Interactive, flexible, great for iteration

🗄️ SQL Production data pipelines
    → Scalable, auditable, version-controlled

🐼 Pandas Complex statistical analysis
    → Rich statistical libraries (scipy, statsmodels)

🗄️ SQL Data > 1GB
    → Memory constraints, let database do the work

🗄️ SQL Real-time dashboards
    → Direct queries, no data movement

🗄️ SQL Ad-hoc business queries
    → Standard language, shareable queries

🐼 Pandas → SQL Machine learning features
    → Prototype in pandas, productionize in SQL

🐼 Pandas Data validation & cleaning
    → Better string/regex operations

🐼 Pandas Time series manipulation
    → Superior datetime handling

🗄️ SQL Joining multiple large tables
    → Optimized query planner



---

## 💡 Part 7: Hybrid Workflows - Best of Both Worlds

The real power comes from combining pandas and SQL seamlessly!

### 7.1 Using SQL for Data Reduction, Pandas for Analysis

In [36]:
# Hybrid approach: Let SQL do the heavy lifting, pandas for fine-tuning

# Step 1: Use SQL to filter and aggregate large data
sql_query = """
SELECT 
    date_only,
    store_id,
    COUNT(DISTINCT customer_id) as unique_customers,
    SUM(revenue) as total_revenue,
    AVG(revenue) as avg_transaction
FROM transactions
WHERE revenue > 50  -- Pre-filter in SQL
GROUP BY date_only, store_id
"""

print("Step 1: SQL for heavy lifting")
show_sql(sql_query)

# Execute and get results
daily_store_metrics = pd.read_sql(sql_query, conn)
print(f"\n✅ Reduced to {len(daily_store_metrics):,} rows\n")

# Step 2: Use pandas for complex transformations
print("Step 2: Pandas for complex analysis")

# Convert to datetime
daily_store_metrics['date_only'] = pd.to_datetime(daily_store_metrics['date_only'])

# Add time-based features
daily_store_metrics['day_of_week'] = daily_store_metrics['date_only'].dt.day_name()
daily_store_metrics['is_weekend'] = daily_store_metrics['date_only'].dt.dayofweek.isin([5, 6])

# Calculate store performance ranking by day
daily_store_metrics['daily_rank'] = daily_store_metrics.groupby('date_only')['total_revenue'].rank(ascending=False)

# Show results
print(daily_store_metrics.head(10))

print("\n💡 Best Practice: Use SQL to reduce data volume, pandas for complex transformations!")

Step 1: SQL for heavy lifting


```sql

SELECT 
    date_only,
    store_id,
    COUNT(DISTINCT customer_id) as unique_customers,
    SUM(revenue) as total_revenue,
    AVG(revenue) as avg_transaction
FROM transactions
WHERE revenue > 50  -- Pre-filter in SQL
GROUP BY date_only, store_id

```


✅ Reduced to 525 rows

Step 2: Pandas for complex analysis
   date_only store_id  unique_customers  total_revenue  avg_transaction  \
0 2024-01-01      CHI                23       15270.54           663.94   
1 2024-01-01      HOU                17       13985.56           822.68   
2 2024-01-01       LA                16        9338.46           583.65   
3 2024-01-01      NYC                17       10206.44           600.38   
4 2024-01-01      PHX                22       10885.96           494.82   
5 2024-01-02      CHI                19       11619.60           611.56   
6 2024-01-02      HOU                16        8424.43           526.53   
7 2024-01-02       LA                16        9202.91           575.18   
8 2024-01-02      NYC                18       11879.36           659.96   
9 2024-01-02      PHX                23       18366.16           798.53   

  day_of_week  is_weekend  daily_rank  
0      Monday       False        1.00  
1      Monday       False        2

### 7.2 Parameterized Queries from Pandas

In [37]:
# Safe parameterized queries - avoid SQL injection!

def get_customer_history(customer_id, min_revenue=0):
    """
    Safely query customer transaction history
    """
    query = """
    SELECT 
        t.transaction_id,
        t.date,
        p.product_name,
        p.category,
        t.revenue
    FROM transactions t
    JOIN products p ON t.product_id = p.product_id
    WHERE t.customer_id = ?
      AND t.revenue > ?
    ORDER BY t.date DESC
    LIMIT 10
    """
    
    # Use parameterized query for safety
    return pd.read_sql(query, conn, params=(customer_id, min_revenue))

# Example usage
customer_data = get_customer_history(customer_id=42, min_revenue=100)
print("🔒 Safe parameterized query result:")
print(customer_data)

print("\n⚠️ NEVER use string formatting for SQL queries - always use parameters!")

🔒 Safe parameterized query result:
   transaction_id                 date product_name     category  revenue
0            9776  2024-04-11 19:45:00  Product_178     Clothing   641.86
1            7728  2024-03-21 11:45:00    Product_5       Sports   936.63
2            2467  2024-01-26 16:30:00  Product_142         Food   601.92
3            1469  2024-01-16 07:00:00   Product_71  Electronics  1163.91
4            1397  2024-01-15 13:00:00  Product_137       Sports   245.34

⚠️ NEVER use string formatting for SQL queries - always use parameters!


### 7.3 Pushing Pandas Operations to SQL

In [38]:
# Sometimes it's better to push operations to the database

# Scenario: Complex filtering that could be done in either tool
stores_of_interest = ['NYC', 'LA']
date_range = ('2024-01-01', '2024-01-07')

print("Approach 1: Load all data, filter in pandas (DON'T DO THIS)")
print("```python")
print("# This loads ALL data into memory first!")
print("df = pd.read_sql('SELECT * FROM transactions', conn)")
print("df_filtered = df[(df['store_id'].isin(stores)) & (df['date'] >= start)]")
print("```\n")

print("Approach 2: Filter in SQL (DO THIS)")
query = f"""
SELECT *
FROM transactions
WHERE store_id IN ({','.join(['?'] * len(stores_of_interest))})
  AND date >= ?
  AND date <= ?
"""

show_sql(query.replace('?', "'store_name'"))

# Execute with parameters
filtered_data = pd.read_sql(
    query, 
    conn, 
    params=stores_of_interest + list(date_range)
)

print(f"\n✅ Loaded only {len(filtered_data):,} relevant rows instead of {len(transactions):,}!")

Approach 1: Load all data, filter in pandas (DON'T DO THIS)
```python
# This loads ALL data into memory first!
df = pd.read_sql('SELECT * FROM transactions', conn)
df_filtered = df[(df['store_id'].isin(stores)) & (df['date'] >= start)]
```

Approach 2: Filter in SQL (DO THIS)


```sql

SELECT *
FROM transactions
WHERE store_id IN ('store_name','store_name')
  AND date >= 'store_name'
  AND date <= 'store_name'

```


✅ Loaded only 205 relevant rows instead of 10,000!


---

## 🎯 Practice Exercises

Now it's your turn! Complete these exercises using BOTH pandas and SQL.

### Exercise 1: Customer Segmentation

Find the top 10% of customers by total spending and analyze their behavior.

In [84]:
# TODO: Your pandas solution here
# Hint: Use quantile() to find the 90th percentile threshold

# pandas_solution = ...
unique_customer = len(transactions['customer_id'].unique())
customers_grouped = transactions.groupby('customer_id')['revenue'].sum()
customers_90th_percentile = customers_grouped[customers_grouped>customers_grouped.quantile(0.9)]

print("🐼 Your pandas solution:")
print(f"Total customers: {unique_customer}.")
print(f"There are {len(customers_90th_percentile)} customers with total spending higher than 90th percentile of revenue (${customers_grouped.quantile(0.9)})")
print(customers_90th_percentile.sort_values(ascending=False))


🐼 Your pandas solution:
Total customers: 1498.
There are 150 customers with total spending higher than 90th percentile of revenue ($6791.655)
customer_id
277    11874.50
700    11816.10
798    11360.77
1257   10669.33
849    10524.02
         ...   
793     6821.31
859     6808.82
1136    6804.27
988     6802.18
1438    6799.39
Name: revenue, Length: 150, dtype: float64


In [90]:
# TODO: Your SQL solution here
# Hint: Use NTILE() or calculate percentiles with window functions

sql_query = """
WITH total_spend_customer AS (
    SELECT 
        customer_id,
        SUM(revenue) as sum_revenue
    FROM transactions
    GROUP BY customer_id
),
top10 AS (
    SELECT
        customer_id, sum_revenue,
        NTILE(10) OVER (ORDER BY sum_revenue) AS decile
    FROM total_spend_customer
)
SELECT
    customer_id, sum_revenue,
    decile as P90_total_spend   
FROM top10
WHERE decile = 10
ORDER BY sum_revenue DESC
"""
print("🗄️ Your SQL solution:")
show_sql(sql_query)

sql_solution = pd.read_sql(sql_query, conn)
print("Output:")
print(sql_solution)

🗄️ Your SQL solution:


```sql

WITH total_spend_customer AS (
    SELECT 
        customer_id,
        SUM(revenue) as sum_revenue
    FROM transactions
    GROUP BY customer_id
),
top10 AS (
    SELECT
        customer_id, sum_revenue,
        NTILE(10) OVER (ORDER BY sum_revenue) AS decile
    FROM total_spend_customer
)
SELECT
    customer_id, sum_revenue,
    decile as P90_total_spend   
FROM top10
WHERE decile = 10
ORDER BY sum_revenue DESC

```

Output:
     customer_id  sum_revenue  P90_total_spend
0            277     11874.50               10
1            700     11816.10               10
2            798     11360.77               10
3           1257     10669.33               10
4            849     10524.02               10
..           ...          ...              ...
144          611      6835.58               10
145          793      6821.31               10
146          859      6808.82               10
147         1136      6804.27               10
148          988      6802.18               10

[149 rows x 3 columns]


### Exercise 2: Cohort Analysis

Calculate retention by customer signup month.

In [ ]:
# TODO: Create a cohort analysis
# 1. Group customers by signup month
# 2. Track their activity in subsequent months
# 3. Calculate retention rates

# Your solution here

In [120]:
print("🐼 Your pandas solution:")
print(f"1) Create signup_month column in customers table.")

customers['signup_month'] = customers['signup_date'].dt.to_period('M')

print(f"2) Left merge transactions table with customers table, on customer_id.")

merged = pd.merge(
    transactions[['transaction_id', 'customer_id', 'date','revenue']],
    customers[['customer_id', 'signup_date', 'signup_month']],
    on = 'customer_id',
    how= 'left'
)

print(f"3) Compute months between signup and transaction date, create a new column months_since_signup.")

merged['months_since_signup']=((merged['date'].dt.year - merged['signup_date'].dt.year)*12 + (merged['date'].dt.month-merged['signup_date'].dt.month)).astype('Int64')

print(f"4) Find active customers: filter out the rows where months_since_signup >= 0, drop duplicates.")

active_customers = merged[merged['months_since_signup']>= 0][['customer_id','signup_month','months_since_signup']].drop_duplicates()

print(f"4a) Group the customer by signup_month and months_since_signup.")
print(f"4b) Find the count of active customers at most recent 'months_since_signup'.")

active_customers_grouped = active_customers.groupby(['signup_month','months_since_signup'])['customer_id'].nunique().reset_index()
active_customers_recent_idx = active_customers_grouped.groupby('signup_month')['months_since_signup'].idxmax()
active_customers_recent = active_customers_grouped.loc[active_customers_recent_idx].rename(columns = {'customer_id':'latest_active_count'})

print(f"5) Find the original count of customers during signup.")
print(f"5a) Merge the customer signup_count to the active customer table.")
print(f"5b) Compute the retention: latest active customer / original signup.")
print(f"\nFinal output:\n")

signup_count = customers.groupby('signup_month')['customer_id'].nunique().reset_index().rename(columns = {'customer_id':'signup_count'})
retention = pd.merge(
    active_customers_recent,
    signup_count,
    on = 'signup_month',
    how = 'left'
)
retention['retention_rate'] = round(retention['latest_active_count'] / retention['signup_count'] * 100, 2)
retention     

🐼 Your pandas solution:
1) Create signup_month column in customers table.
2) Left merge transactions table with customers table, on customer_id.
3) Compute months between signup and transaction date, create a new column months_since_signup.
4) Find active customers: filter out the rows where months_since_signup >= 0, drop duplicates.
4a) Group the customer by signup_month and months_since_signup.
4b) Find the count of active customers at most recent 'months_since_signup'.
5) Find the original count of customers during signup.
5a) Merge the customer signup_count to the active customer table.
5b) Compute the retention: latest active customer / original signup.

Final output:



signup_month  months_since_signup  latest_active_count  signup_count  \
0       2023-01                   15                   65           124   
1       2023-02                   14                   65           112   
2       2023-03                   13                   67           124   
3       2023-04                   12                   67           120   
4       2023-05                   11                   64           124   
5       2023-06                   10                   64           120   
6       2023-07                    9                   79           124   
7       2023-08                    8                   75           124   
8       2023-09                    7                   70           120   
9       2023-10                    6                   71           124   
10      2023-11                    5                   67           120   
11      2023-12                    4                   75           124   
12      2024-01                    3                   24            40   

    retention_rate  
0            52.42  
1            58.04  
2            54.03  
3            55.83  
4            51.61  
5            53.33  
6            63.71  
7            60.48  
8            58.33  
9            57.26  
10           55.83  
11           60.48  
12           60.00

In [121]:
# SQL solution

sql_query_ex2 = """
WITH base AS (
    SELECT 
        COUNT (DISTINCT t.customer_id) AS active_customers,
        date(t.date, 'start of month') AS transaction_month,
        date(c.signup_date, 'start of month') AS signup_month,
        (CAST(strftime('%Y', t.date) AS INT) * 12 + CAST(strftime('%m', t.date) AS INT)
      - CAST(strftime('%Y', c.signup_date) AS INT) * 12 - CAST(strftime('%m', c.signup_date) AS INT)    
        ) AS months_since_signup
    FROM transactions t
    LEFT JOIN customers c 
    ON t.customer_id = c.customer_id
    GROUP BY signup_month, months_since_signup
),
recent AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY signup_month ORDER BY months_since_signup DESC) AS recent_flag
    FROM base
),
signup AS (
    SELECT 
        COUNT(*) AS signup_count,
        date(signup_date, 'start of month') AS signup_month
    FROM customers
    GROUP BY strftime('%Y-%m', signup_date)
)
SELECT 
    r.signup_month, r.transaction_month as latest_transaction_month, r.months_since_signup, r.active_customers,
    s.signup_count,
    ROUND(100.0 * r.active_customers / NULLIF(s.signup_count,0), 2) AS retention_percentage
FROM recent r
LEFT JOIN signup s ON r.signup_month = s.signup_month
WHERE recent_flag = 1
"""
print("🗄️ Your SQL solution:")
show_sql(sql_query_ex2)
sql_ex2_result = pd.read_sql(sql_query_ex2,conn)
sql_ex2_result

🗄️ Your SQL solution:


```sql

WITH base AS (
    SELECT 
        COUNT (DISTINCT t.customer_id) AS active_customers,
        date(t.date, 'start of month') AS transaction_month,
        date(c.signup_date, 'start of month') AS signup_month,
        (CAST(strftime('%Y', t.date) AS INT) * 12 + CAST(strftime('%m', t.date) AS INT)
      - CAST(strftime('%Y', c.signup_date) AS INT) * 12 - CAST(strftime('%m', c.signup_date) AS INT)    
        ) AS months_since_signup
    FROM transactions t
    LEFT JOIN customers c 
    ON t.customer_id = c.customer_id
    GROUP BY signup_month, months_since_signup
),
recent AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY signup_month ORDER BY months_since_signup DESC) AS recent_flag
    FROM base
),
signup AS (
    SELECT 
        COUNT(*) AS signup_count,
        date(signup_date, 'start of month') AS signup_month
    FROM customers
    GROUP BY strftime('%Y-%m', signup_date)
)
SELECT 
    r.signup_month, r.transaction_month as latest_transaction_month, r.months_since_signup, r.active_customers,
    s.signup_count,
    ROUND(100.0 * r.active_customers / NULLIF(s.signup_count,0), 2) AS retention_percentage
FROM recent r
LEFT JOIN signup s ON r.signup_month = s.signup_month
WHERE recent_flag = 1

```

signup_month latest_transaction_month  months_since_signup  \
0    2023-01-01               2024-04-01                   15   
1    2023-02-01               2024-04-01                   14   
2    2023-03-01               2024-04-01                   13   
3    2023-04-01               2024-04-01                   12   
4    2023-05-01               2024-04-01                   11   
5    2023-06-01               2024-04-01                   10   
6    2023-07-01               2024-04-01                    9   
7    2023-08-01               2024-04-01                    8   
8    2023-09-01               2024-04-01                    7   
9    2023-10-01               2024-04-01                    6   
10   2023-11-01               2024-04-01                    5   
11   2023-12-01               2024-04-01                    4   
12   2024-01-01               2024-04-01                    3   

    active_customers  signup_count  retention_percentage  
0                 65           124                 52.42  
1                 65           112                 58.04  
2                 67           124                 54.03  
3                 67           120                 55.83  
4                 64           124                 51.61  
5                 64           120                 53.33  
6                 79           124                 63.71  
7                 75           124                 60.48  
8                 70           120                 58.33  
9                 71           124                 57.26  
10                67           120                 55.83  
11                75           124                 60.48  
12                24            40                 60.00

### Exercise 3: Product Affinity

Find which products are frequently bought together.

In [ ]:
# TODO: Implement market basket analysis
# Find products that appear in the same transactions

# Your solution here

In [31]:
print("🐼 Your pandas solution:")

print(f"1) Find transactions on the same date by the same customers.")

transactions['date_only']=pd.to_datetime(transactions['date_only'])
same_day_transactions = transactions.groupby(['date_only','customer_id'])['product_id'].nunique().reset_index()
same_day_transactions = same_day_transactions.rename(columns={'product_id':'unique_product_count'})

print(f"2) Merge transactions with same_day_transactions, filter to rows where unique_product_count is > 1.")

product_purchase = pd.merge(
    transactions[['date_only','customer_id', 'product_id', 'quantity']],
    same_day_transactions[['date_only', 'customer_id', 'unique_product_count']],
    on = ['date_only','customer_id'],
    how = 'left')
product_purchase_multiple = product_purchase[product_purchase['unique_product_count']>1]
product_purchase_multiple = product_purchase_multiple.sort_values(['date_only','customer_id']).drop('unique_product_count', axis=1)

print(f"3) Group by date and customer_id, aggregate the list of product_id.")

product_purchase_grouped = product_purchase_multiple.groupby(['date_only','customer_id'])['product_id'].agg(list)

print(f"\nOutput from step 2:\n")
print(product_purchase_multiple)

print(f"\nOutput from step 3:\n")
print(product_purchase_grouped)

🐼 Your pandas solution:
1) Find transactions on the same date by the same customers.
2) Merge transactions with same_day_transactions, filter to rows where unique_product_count is > 1.
3) Group by date and customer_id, aggregate the list of product_id.

Output from step 2:

      date_only  customer_id  product_id  quantity
24   2024-01-01          956         158         4
64   2024-01-01          956         165         2
40   2024-01-01          958          47         2
42   2024-01-01          958          66         4
3    2024-01-01         1295         158         3
...         ...          ...         ...       ...
9931 2024-04-13         1089         141         1
9913 2024-04-13         1130         112         1
9978 2024-04-13         1130         166         1
9959 2024-04-13         1154         142         3
9976 2024-04-13         1154         125         2

[601 rows x 4 columns]

Output from step 3:

date_only   customer_id
2024-01-01  956            [158, 165]
     

In [32]:
# SQL solution
# option 1: output table is similar to panda's output step 2

sql_query_ex4 = """
WITH base AS (
    SELECT 
        date_only, customer_id, product_id
    FROM transactions
),
same_day AS (
    SELECT
        date_only, customer_id,
        COUNT(DISTINCT product_id) AS unique_product_count    
    FROM base
    GROUP BY date_only, customer_id
),
multiple_purchase AS (
    SELECT 
        b.date_only, b.customer_id, b.product_id
    FROM base b
    JOIN same_day s 
    ON b.date_only = s.date_only AND b.customer_id = s.customer_id
    WHERE s.unique_product_count > 1
)
SELECT *
FROM multiple_purchase
ORDER BY date_only, customer_id, product_id
"""
show_sql(sql_query_ex4)
sql_result_ex4 = pd.read_sql(sql_query_ex4,conn)
sql_result_ex4

```sql

WITH base AS (
    SELECT 
        date_only, customer_id, product_id
    FROM transactions
),
same_day AS (
    SELECT
        date_only, customer_id,
        COUNT(DISTINCT product_id) AS unique_product_count    
    FROM base
    GROUP BY date_only, customer_id
),
multiple_purchase AS (
    SELECT 
        b.date_only, b.customer_id, b.product_id
    FROM base b
    JOIN same_day s 
    ON b.date_only = s.date_only AND b.customer_id = s.customer_id
    WHERE s.unique_product_count > 1
)
SELECT *
FROM multiple_purchase
ORDER BY date_only, customer_id, product_id

```

date_only  customer_id  product_id
0    2024-01-01          956         158
1    2024-01-01          956         165
2    2024-01-01          958          47
3    2024-01-01          958          66
4    2024-01-01         1295          18
..          ...          ...         ...
596  2024-04-13         1089         141
597  2024-04-13         1130         112
598  2024-04-13         1130         166
599  2024-04-13         1154         125
600  2024-04-13         1154         142

[601 rows x 3 columns]

In [33]:
# option 2: output table is similar to panda's output step 3
sql_query_ex3 = """
WITH base AS (
    SELECT 
        date_only, customer_id, product_id
    FROM transactions
),
same_day AS (
    SELECT
        date_only, customer_id,
        COUNT(DISTINCT product_id) AS unique_product_count    
    FROM base
    GROUP BY date_only, customer_id
),
multiple_purchase AS (
    SELECT 
        b.date_only, b.customer_id, b.product_id
    FROM base b
    JOIN same_day s 
    ON b.date_only = s.date_only AND b.customer_id = s.customer_id
    WHERE s.unique_product_count > 1
)
SELECT
    date_only,
    customer_id,
    group_concat(product_id, ',') AS product_ids
FROM multiple_purchase
GROUP BY date_only, customer_id
ORDER BY date_only, customer_id
"""
show_sql(sql_query_ex3)
sql_result_ex3 = pd.read_sql(sql_query_ex3,conn)
sql_result_ex3

```sql

WITH base AS (
    SELECT 
        date_only, customer_id, product_id
    FROM transactions
),
same_day AS (
    SELECT
        date_only, customer_id,
        COUNT(DISTINCT product_id) AS unique_product_count    
    FROM base
    GROUP BY date_only, customer_id
),
multiple_purchase AS (
    SELECT 
        b.date_only, b.customer_id, b.product_id
    FROM base b
    JOIN same_day s 
    ON b.date_only = s.date_only AND b.customer_id = s.customer_id
    WHERE s.unique_product_count > 1
)
SELECT
    date_only,
    customer_id,
    group_concat(product_id, ',') AS product_ids
FROM multiple_purchase
GROUP BY date_only, customer_id
ORDER BY date_only, customer_id

```

date_only  customer_id product_ids
0    2024-01-01          956     158,165
1    2024-01-01          958       47,66
2    2024-01-01         1295      158,18
3    2024-01-02          231     175,126
4    2024-01-02          503       94,73
..          ...          ...         ...
293  2024-04-12          700     142,140
294  2024-04-13          317      146,69
295  2024-04-13         1089       7,141
296  2024-04-13         1130     112,166
297  2024-04-13         1154     142,125

[298 rows x 3 columns]

---

## 🎓 Key Takeaways

1. **Mental Model Mapping**:
   - `df.groupby()` → `GROUP BY`
   - `df.merge()` → `JOIN`
   - `df['col'].rank()` → `RANK() OVER()`
   - `df['col'].shift()` → `LAG()/LEAD()`

2. **When to Use SQL**:
   - Data is in a database (avoid loading unnecessary data)
   - Need to share queries with non-Python users
   - Production pipelines requiring audit trails
   - Working with data larger than memory

3. **When to Use Pandas**:
   - Exploratory data analysis
   - Complex statistical operations
   - Data cleaning and string manipulation
   - Visualization preparation

4. **Best Practices**:
   - Use SQL to reduce data volume first
   - Always use parameterized queries
   - Think in sets (SQL) vs iterations (pandas)
   - Document complex queries with CTEs
   - Profile performance for large datasets

5. **Hybrid Approach**:
   - SQL for extraction and reduction
   - Pandas for transformation and analysis
   - SQL for productionization

---

## 🚀 Next Steps

In the next notebook, we'll dive deeper into:
- Data warehouse design patterns
- Star and snowflake schemas
- Optimizing query performance
- Working with cloud data warehouses

Remember: **You don't choose pandas OR SQL - you master BOTH!** 🎯

In [34]:
# Clean up
conn.close()
print("✅ Database connection closed. Great work!")

✅ Database connection closed. Great work!
